In [4]:
import os

# Read the content from the file
file_path = './all_transcription.txt'
output_dir = './transcriptions'
chunk_size = 500  # Number of words per chunk, adjust as needed

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Read the entire content of the transcription file
# Read the entire content of the transcription file with utf-8 encoding
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into words
words = text.split()
num_words = len(words)
chunk_count = (num_words // chunk_size) + 1

# Divide the text into chunks and save each chunk
for i in range(chunk_count):
    start = i * chunk_size
    end = start + chunk_size
    chunk_words = words[start:end]
    
    # Join words to make a chunk of text
    chunk_text = ' '.join(chunk_words)
    
    # Save each chunk as a new file
    chunk_file_path = os.path.join(output_dir, f'chunk_{i+1}.txt')
    with open(chunk_file_path, 'w',encoding='utf-8') as chunk_file:
        chunk_file.write(chunk_text)
    
    print(f'Saved chunk {i+1} to {chunk_file_path}')


Saved chunk 1 to ./transcriptions\chunk_1.txt
Saved chunk 2 to ./transcriptions\chunk_2.txt
Saved chunk 3 to ./transcriptions\chunk_3.txt
Saved chunk 4 to ./transcriptions\chunk_4.txt
Saved chunk 5 to ./transcriptions\chunk_5.txt
Saved chunk 6 to ./transcriptions\chunk_6.txt
Saved chunk 7 to ./transcriptions\chunk_7.txt
Saved chunk 8 to ./transcriptions\chunk_8.txt
Saved chunk 9 to ./transcriptions\chunk_9.txt
Saved chunk 10 to ./transcriptions\chunk_10.txt
Saved chunk 11 to ./transcriptions\chunk_11.txt
Saved chunk 12 to ./transcriptions\chunk_12.txt
Saved chunk 13 to ./transcriptions\chunk_13.txt
Saved chunk 14 to ./transcriptions\chunk_14.txt
Saved chunk 15 to ./transcriptions\chunk_15.txt
Saved chunk 16 to ./transcriptions\chunk_16.txt
Saved chunk 17 to ./transcriptions\chunk_17.txt
Saved chunk 18 to ./transcriptions\chunk_18.txt
Saved chunk 19 to ./transcriptions\chunk_19.txt
Saved chunk 20 to ./transcriptions\chunk_20.txt
Saved chunk 21 to ./transcriptions\chunk_21.txt
Saved chun

In [1]:
import os

def move_until_period(current_chunk, next_chunk):
    # Check if current chunk ends without a period
    if not current_chunk.endswith('.'):
        # Find the first period in the next chunk
        period_index = next_chunk.find('.')
        if period_index != -1:
            # Move the content from the start of next_chunk up to and including the first period
            additional_text = next_chunk[:period_index + 1]
            # Append this to the current chunk and update next_chunk
            current_chunk += additional_text
            next_chunk = next_chunk[period_index + 1:].lstrip()  # Remove the moved part from next_chunk and strip leading whitespace
    return current_chunk, next_chunk

def complete_sentences(folder_path):
    # Get a sorted list of all text files in the folder
    chunk_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.txt')])
    
    chunks = []
    for file in chunk_files:
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r',encoding='utf-8') as f:
            chunks.append(f.read().strip())
    
    completed_chunks = []
    
    # Process each chunk and move part of next chunk if needed
    for i in range(len(chunks) - 1):
        current_chunk = chunks[i]
        next_chunk = chunks[i + 1]
        
        # Move part of next_chunk until the first period if current_chunk doesn't end with one
        current_chunk, next_chunk = move_until_period(current_chunk, next_chunk)
        
        # Add the modified current_chunk to completed_chunks
        completed_chunks.append(current_chunk)
        
        # Update the next chunk in the chunks list for the next iteration
        chunks[i + 1] = next_chunk
    
    # Append the last chunk as it has no next chunk to compare
    completed_chunks.append(chunks[-1])
    
    # Write the completed text back to files in a new folder 'completed_transcriptions'
    output_folder = os.path.join("./Refined")
    os.makedirs(output_folder, exist_ok=True)
    
    for idx, text in enumerate(completed_chunks):
        output_file = os.path.join(output_folder, f'completed_chunk_{idx+1}.txt')
        with open(output_file, 'w',encoding='utf-8') as f:
            f.write(text)
    
    print("Text completion and cleanup done! Check the 'completed_transcriptions' folder.")

# Folder path containing transcription files
folder_path = "transcriptions"
complete_sentences(folder_path)


Text completion and cleanup done! Check the 'completed_transcriptions' folder.


In [3]:
import os
import openai

# Set up OpenAI API with your credentials
openai.api_key = "pk-XaEUxTeyesYpIltKXeimYKZjDYJhrfWyoKXRwITKKYAhfrJt"
openai.base_url = "https://api.pawan.krd/pai-001/v1/"

def refine_text(text):
    # Call the OpenAI API to refine each chunk of text
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Please summarize the following paragraph into a concise paragraph that includes all essential details, including key statistics and numbers. The output should be refined plain text only. It should contain all inportant details in it , Here is the paragraph:'{text}'"},
        ],
    )
    # Extract the refined text from the API response
    refined_text = response.choices[0].message.content
    return refined_text


def refine_completed_files(folder_path):
    # Path to folder containing completed text files
    completed_folder = os.path.join("./Refined")
    refined_folder = os.path.join("./Final_confirm")
    
    # Ensure the refined folder exists
    os.makedirs(refined_folder, exist_ok=True)
    
    # Process each completed file in the folder
    for file_name in sorted(os.listdir(completed_folder)):
        if file_name.endswith('.txt'):
            # Read the content of each completed file
            file_path = os.path.join(completed_folder, file_name)
            with open(file_path, 'r' ,encoding='utf-8') as file:
                content = file.read()
            
            # Refine the content using OpenAI API
            refined_content = refine_text(content)
            
            # Write the refined content to a new file in the refined folder
            refined_file_path = os.path.join(refined_folder, f'refined_{file_name}')
            with open(refined_file_path, 'w',encoding='utf-8') as refined_file:
                refined_file.write(refined_content)
    
    print("Refinement done! Check the 'refined_transcriptions' folder.")

# Folder path containing the 'completed_transcriptions' folder
folder_path = "transcriptions"
refine_completed_files(folder_path)

Refinement done! Check the 'refined_transcriptions' folder.


In [4]:
import os

def combine_files(input_folder, output_file):
    # Open the output file in write mode
    with open(output_file, 'w', encoding='utf-8') as outfile:
        # Iterate through each file in the input folder
        for file_name in sorted(os.listdir(input_folder)):
            if file_name.endswith('.txt'):
                file_path = os.path.join(input_folder, file_name)
                # Read content from each file and write to the output file
                with open(file_path, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read() + "\n\n")  # Add separation between files
    print(f"All files combined into {output_file}.")

# Define the folder containing refined text files and the output file name
input_folder = "./Final_confirm"
output_file = "./final_combined_output.txt"

combine_files(input_folder, output_file)


All files combined into ./final_combined_output.txt.
